<a href="https://colab.research.google.com/github/Remo6985/AOL-AI/blob/main/AOL_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
from sklearn.model_selection import KFold
import os
import shutil
from glob import glob
import numpy as np

In [ ]:
# Mount Google Drive
drive.mount('/content/gdrive')

# Unzip dataset
!scp '/content/gdrive/My Drive/AOL AI/AOL AI.zip' '/content/AOL AI.zip'
!unzip '/content/AOL AI.zip' -d '/content/'

In [ ]:
# Install YOLOv8
!pip install ultralytics

from ultralytics import YOLO

In [5]:
# Define dataset path
dataset_path = "/content/AOL AI/"
num_folds = 3  # Number of folds for cross-validation

# Gather all class folders from train/
class_folders = [os.path.join(dataset_path, "train", class_dir) for class_dir in os.listdir(os.path.join(dataset_path, "train")) if os.path.isdir(os.path.join(dataset_path, "train", class_dir))]

# Gather all images from train/
all_images = []
for class_folder in class_folders:
    class_images = glob(os.path.join(class_folder, "*"))  # All images in this class folder
    all_images.extend(class_images)

print(f"Total images: {len(all_images)}")
print(f"Classes: {[os.path.basename(folder) for folder in class_folders]}")

Total images: 950
Classes: ['Ectomorph', 'Endomorph', 'Mesomorph']


In [6]:
# Set up K-Fold Cross-Validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

fold = 1
fold_accuracies = []  # To store accuracy for each fold

In [ ]:
for train_idx, val_idx in kf.split(all_images):
    print(f"Processing Fold {fold}...")

    # Create directories for this fold
    fold_dir = f"/content/AOL_AI_fold_{fold}"
    train_dir = os.path.join(fold_dir, "train")
    val_dir = os.path.join(fold_dir, "val")
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    # Split data into train and validation sets
    train_images = [all_images[i] for i in train_idx]
    val_images = [all_images[i] for i in val_idx]

    # Move train images to train/ preserving class structure
    for img in train_images:
        class_name = os.path.basename(os.path.dirname(img))  # Extract class name from folder structure
        class_train_dir = os.path.join(train_dir, class_name)
        os.makedirs(class_train_dir, exist_ok=True)
        shutil.copy(img, class_train_dir)

    # Move validation images to val/ preserving class structure
    for img in val_images:
        class_name = os.path.basename(os.path.dirname(img))  # Extract class name from folder structure
        class_val_dir = os.path.join(val_dir, class_name)
        os.makedirs(class_val_dir, exist_ok=True)
        shutil.copy(img, class_val_dir)

    # Train YOLOv8 Model
    model = YOLO("yolov8n-cls.pt")  # Classification model
    fold_results = model.train(
        data='/content/AOL AI/',  # Pass the YAML file path
        epochs=100,
        imgsz=640,
        batch=16,
        patience=10,  # Early stopping
        optimizer="SGD",
        lr0=0.01,
        weight_decay=1e-4,
        augment=True,
    )

    # Extract accuracy for the current fold
    accuracy = fold_results.top1  # Replace with the correct metric key
    fold_accuracies.append(accuracy)

    print(f"Fold {fold} Accuracy: {accuracy:.2f}")
    fold += 1


In [11]:
# Compute mean and standard deviation of accuracies
mean_accuracy = np.mean(fold_accuracies)
std_accuracy = np.std(fold_accuracies)

# Print aggregated results
print(f"Cross-Validation Accuracies: {fold_accuracies}")
print(f"Mean Accuracy: {mean_accuracy:.2f}")
print(f"Standard Deviation: {std_accuracy:.2f}")


Cross-Validation Accuracies: [0.9032257795333862, 0.9032257795333862, 0.9032257795333862]
Mean Accuracy: 0.90
Standard Deviation: 0.00


In [12]:
!scp -r /content/runs '/content/gdrive/My Drive/AOL AI'